In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

x_train = train_data.iloc[:, 1:].values / 255.0  # Normalize pixel values, skipping the label column
y_train = train_data.iloc[:, 0].values  # Labels are the first column

x_test = test_data.values / 255.0  # Normalize pixel values

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# One-hot encode the labels for training
y_train = to_categorical(y_train, 10)

cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

cnn_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

cnn_model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

train_loss, train_acc = cnn_model.evaluate(x_train, y_train, verbose=2)
print(f"CNN Training accuracy: {train_acc:.4f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 33s 59ms/step - accuracy: 0.8499 - loss: 0.4921 - val_accuracy: 0.9758 - val_loss: 0.0764
Epoch 2/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 42s 62ms/step - accuracy: 0.9822 - loss: 0.0595 - val_accuracy: 0.9831 - val_loss: 0.0558
Epoch 3/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 38s 58ms/step - accuracy: 0.9881 - loss: 0.0388 - val_accuracy: 0.9864 - val_loss: 0.0429
Epoch 4/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 41s 57ms/step - accuracy: 0.9912 - loss: 0.0261 - val_accuracy: 0.9849 - val_loss: 0.0471
Epoch 5/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 41s 58ms/step - accuracy: 0.9940 - loss: 0.0199 - val_accuracy: 0.9881 - val_loss: 0.0414
Epoch 6/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 41s 57ms/step - accuracy: 0.9961 - loss: 0.0130 - val_accuracy: 0.9867 - val_loss: 0.0468
Epoch 7/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 32s 61ms/step - accuracy: 0.9961 - loss: 0.0127 - val_accuracy: 0.9887 - val_loss: 0.0435
Epoch 8/10
525/525 ━━━━━━━━━━━━━━━━━━━━ 30s 58ms/step - accuracy: 0.9969 - loss: 0.0090 - 

In [ ]:
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Apply PCA to reduce the dimensions
pca = PCA(n_components=50)
x_train_pca = pca.fit_transform(x_train_flat)
x_test_pca = pca.transform(x_test_flat)

x_train_split, x_val_split, y_train_split, y_val_split = train_test_split(x_train_pca, y_train, test_size=0.2, random_state=42)

classifiers = {
    "KNN": KNeighborsClassifier(),
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier()
}

# Train and evaluate each model
model_accuracies = {}

for name, clf in classifiers.items():
    y_train_split_1d = np.argmax(y_train_split, axis=1)
    y_val_split_1d = np.argmax(y_val_split, axis=1)
    clf.fit(x_train_split, y_train_split_1d)
    y_pred = clf.predict(x_val_split)
    accuracy = accuracy_score(y_val_split_1d, y_pred)
    model_accuracies[name] = accuracy

# Accuracy of each model
for name, accuracy in model_accuracies.items():
    print(f"{name} Model Accuracy: {accuracy:.4f}")
print(f"\nCNN Accuracy: {train_acc:.4f}\n")

best_model = "CNN" if train_acc > max(model_accuracies.values()) else max(model_accuracies, key=model_accuracies.get)
print(f"The best model is: {best_model}")

KNN Model Accuracy: 0.9726
SVM Model Accuracy: 0.9777
Decision Tree Model Accuracy: 0.8274
Random Forest Model Accuracy: 0.9487

CNN Accuracy: 0.9952

The best model is: CNN


In [ ]:
import numpy as np
import pandas as pd
from google.colab import files

y_pred_cnn = cnn_model.predict(x_test)

# Convert the predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred_cnn, axis=1)

output_df = pd.DataFrame({
    'ImageId': np.arange(1, len(y_pred_labels) + 1),
    'Label': y_pred_labels
})

output_file = 'mnist_predictions_cnn.csv'
output_df.to_csv(output_file, index=False)

files.download(output_file)

875/875 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>